Used libraries:

In [37]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
import optuna
import pickle

## Preprocessing

In [151]:
# Import dataset
train_df = pd.read_csv('/Users/kehindeslaptop/Code/projects/home-price-prediction/backend/data/training-housing-dataset.csv', index_col=0)

In [152]:
train_df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus,total_rooms,area_per_room,amenity_count
0,1.000000,0.553672,4,2,2,1,0,0,0,1,1,1,1,6,0.243257,2
1,0.990741,0.209040,4,2,2,1,0,0,1,0,2,0,2,6,0.083530,2
2,0.990741,0.694915,3,2,2,1,0,0,0,0,0,1,1,5,0.390416,1
3,0.972222,0.491525,4,1,2,1,0,1,0,0,2,0,1,5,0.277298,2
4,0.972222,0.559322,4,2,2,1,1,1,0,1,1,1,0,6,0.245876,4


In [153]:
# Scaling Data
scaled_columns = ["price", "area", "area_per_room"]
scaler = MinMaxScaler()
train_df[scaled_columns] = scaler.fit_transform(train_df[scaled_columns])

In [154]:
# Creating training and testing datasets
X = train_df.drop(columns="price")
Y = train_df['price']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [155]:
# Creating objective function for model optimization

def objective_elastic(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-4, 1e1)
    l1_ratio = trial.suggest_uniform('l1_ratio', 0.0, 1.0)
    
    # Creating, training and predicting Elastic Net model with suggested hyperparams
    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    mse=mean_squared_error(Y_test, y_pred)
    
    return mse

In [156]:
# Creating study and optimizing
study = optuna.create_study(direction='minimize') # To minimize MSE
study.optimize(objective_elastic, n_trials=100)

[I 2024-10-04 01:56:38,721] A new study created in memory with name: no-name-a679144d-eb57-4053-9164-03fad25a0ab5
/var/folders/12/9kkk6tqs47d4103npslj25_40000gn/T/ipykernel_48697/973572366.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-4, 1e1)
/var/folders/12/9kkk6tqs47d4103npslj25_40000gn/T/ipykernel_48697/973572366.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  l1_ratio = trial.suggest_uniform('l1_ratio', 0.0, 1.0)
[I 2024-10-04 01:56:38,727] Trial 0 finished with value: 0.021406009229664024 and parameters: {'alpha': 0.002428397539488765, 'l1_ratio': 0.831491380556415}. Best is trial 0 with value: 0.021406009229664024.
/var/folders

In [157]:
print('Best parameters found: ', study.best_params)
print("Best MSE achieved", study.best_value)

Best parameters found:  {'alpha': 0.00010748706156684767, 'l1_ratio': 0.9320403160390891}
Best MSE achieved 0.020590098776505156


In [158]:
# Training final model with best parameters
best_params = study.best_params
final_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], random_state=42)
# Training model on entire set
final_model.fit(X_train, Y_train)

# Model Evaluation
y_pred = final_model.predict(X_test)
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print(f"Test MSE with optimized hyperparameters: {mse}")


Test MSE with optimized hyperparameters: 0.020590098776505156


In [159]:
r2 = r2_score(Y_test, y_pred)
print(f"Test r2 with optimized hyperparameters: {r2}")

Test r2 with optimized hyperparameters: 0.671722338027005


In [161]:
# Saving model for use with website

with open('elastic_net_model.pkl', 'wb') as file:
    pickle.dump(final_model, file)